<a href="https://colab.research.google.com/github/kywch/geo-colab/blob/master/SafeGraph-filter-places.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
colab_requirements = [
    "apt install python3-rtree=0.8.3+ds-1", # rtree v0.8.3
    "pip install geopandas==0.7.0" # geopandas v0.7.0 
]

In [9]:
# following the post, 'Making Jupyter notebooks Google Colab ready'
# https://timsainburg.com/google%20colab.html

import os, sys, subprocess

def run_subprocess_command(cmd):
    # run the command
    process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE)
    # print the output
    for line in process.stdout:
        print(line.decode().strip())
        
if 'COLAB_GPU' in os.environ: # a quick and dirty way to check whether this is COLAB
    for i in colab_requirements:
        run_subprocess_command(i)

Reading package lists...
Building dependency tree...
Reading state information...
python3-rtree is already the newest version (0.8.3+ds-1).
The following package was automatically installed and is no longer required:
libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [20]:
import pandas as pd

import geopandas as gpd
from fiona.crs import from_epsg
from shapely.geometry import Point

from glob import glob
import time

import os
from shutil import copyfile

## Load the Places dataset from Google Drive

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [12]:
!ls drive/My\ Drive/SafeGraph/places

brand_info.csv						  core_poi-part4.csv.gz
CorePlacesMay2020Release-CORE_POI-2020_04-2020-05-06.zip  core_poi-part5.csv.gz
core_poi-part1.csv.gz					  README.txt
core_poi-part2.csv.gz					  SquareFeet.csv.gz
core_poi-part3.csv.gz


In [13]:
# grab the SafeGraph core places
files = glob("drive/My Drive/SafeGraph/places/core*.csv.gz")
print(files)

['drive/My Drive/SafeGraph/places/core_poi-part4.csv.gz', 'drive/My Drive/SafeGraph/places/core_poi-part5.csv.gz', 'drive/My Drive/SafeGraph/places/core_poi-part1.csv.gz', 'drive/My Drive/SafeGraph/places/core_poi-part3.csv.gz', 'drive/My Drive/SafeGraph/places/core_poi-part2.csv.gz']


In [21]:
# copy these files from Google Drive to colab disk
os.mkdir('sg_data')
for f in files:
  copyfile(f, 'sg_data/' + os.path.basename(f))

In [24]:
#!ls sg_data
files = glob("sg_data/core*.csv.gz")
print(files)

['sg_data/core_poi-part3.csv.gz', 'sg_data/core_poi-part2.csv.gz', 'sg_data/core_poi-part1.csv.gz', 'sg_data/core_poi-part5.csv.gz', 'sg_data/core_poi-part4.csv.gz']


In [29]:
place_df = pd.read_csv(files[0])
for f in files[1:]:
    t = time.time()
    tmp_pd2 = pd.read_csv(f)
    place_df = place_df.append(tmp_pd2, ignore_index = True)
    print(f + ', new_len: ', str(len(place_df)) + ', proc time: ', str(time.time() - t))

sg_data/core_poi-part2.csv.gz, new_len:  2141464, proc time:  9.577615737915039
sg_data/core_poi-part1.csv.gz, new_len:  3210953, proc time:  10.44595193862915
sg_data/core_poi-part5.csv.gz, new_len:  4220022, proc time:  10.723469972610474
sg_data/core_poi-part4.csv.gz, new_len:  5284986, proc time:  12.158286809921265


In [30]:
place_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5284986 entries, 0 to 5284985
Data columns (total 18 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   safegraph_place_id         object 
 1   parent_safegraph_place_id  object 
 2   location_name              object 
 3   safegraph_brand_ids        object 
 4   brands                     object 
 5   top_category               object 
 6   sub_category               object 
 7   naics_code                 float64
 8   latitude                   float64
 9   longitude                  float64
 10  street_address             object 
 11  city                       object 
 12  region                     object 
 13  postal_code                int64  
 14  iso_country_code           object 
 15  phone_number               float64
 16  open_hours                 object 
 17  category_tags              object 
dtypes: float64(4), int64(1), object(13)
memory usage: 725.8+ MB


In [31]:
place_df.head()

,safegraph_place_id,parent_safegraph_place_id,location_name,safegraph_brand_ids,brands,top_category,sub_category,naics_code,latitude,longitude,street_address,city,region,postal_code,iso_country_code,phone_number,open_hours,category_tags
0,sg:0001f7d539e7458f882539e57cd9ecae,NaN,Silver Creek Church of Nazarene,NaN,NaN,Religious Organizations,Religious Organizations,813110.0,36.283962,-95.812324,9000 N 145th East Ave,Owasso,OK,74055,US,1.918272e+10,NaN,NaN
1,sg:0011b49da3e3407fadb93e96953a1c0b,sg:27197b07189c4282abacd319a764d26f,Jeffrey Rippon Ameriprise Financial Services,NaN,NaN,Other Financial Investment Activities,Investment Advice,523930.0,40.768929,-111.888847,60 E South Temple Ste 950,Salt Lake City,UT,84111,US,1.801602e+10,"{ ""Mon"": [[""8:00"", ""17:00""]], ""Tue"": [[""8:00"",...",NaN
2,sg:002aa7b42ab745f28ba25c3bb524f195,NaN,Tampa's Best Dog Treats and Chews,NaN,NaN,Other Miscellaneous Store Retailers,Pet and Pet Supplies Stores,453910.0,28.061555,-82.502928,12651 N Dale Mabry Hwy Unit 273651,Tampa,FL,33688,US,NaN,"{ ""Mon"": [[""8:00"", ""18:00""]], ""Tue"": [[""8:00"",...",NaN
3,sg:0039682ceac447d594d95d75727da59d,NaN,Dauer Museum Of Classic Cars,NaN,NaN,"Museums, Historical Sites, and Similar Institu...",Museums,712110.0,26.163697,-80.216703,4850 W Oakland Park Blvd,Lauderdale Lakes,FL,33313,US,1.954739e+10,NaN,NaN
4,sg:005a813d80d24f7bbc888f76d31a031d,NaN,Hamilton's Produce,NaN,NaN,Grocery Stores,Supermarkets and Other Grocery (except Conveni...,445110.0,30.823265,-88.362247,13451 Moffett Rd,Wilmer,AL,36587,US,1.251645e+10,NaN,NaN


In [68]:
# translate latitude and longitude to geopandas
place_gdf = gpd.GeoDataFrame(
    place_df, geometry = gpd.points_from_xy(place_df.longitude, place_df.latitude),
    crs = "epsg:4326" # https://geopandas.readthedocs.io/en/v0.6.0/projections.html#coordinate-reference-systems
).to_crs(epsg=3528) # https://spatialreference.org/ref/epsg/3528/

In [69]:
place_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 5284986 entries, 0 to 5284985
Data columns (total 19 columns):
 #   Column                     Dtype   
---  ------                     -----   
 0   safegraph_place_id         object  
 1   parent_safegraph_place_id  object  
 2   location_name              object  
 3   safegraph_brand_ids        object  
 4   brands                     object  
 5   top_category               object  
 6   sub_category               object  
 7   naics_code                 float64 
 8   latitude                   float64 
 9   longitude                  float64 
 10  street_address             object  
 11  city                       object  
 12  region                     object  
 13  postal_code                int64   
 14  iso_country_code           object  
 15  phone_number               float64 
 16  open_hours                 object  
 17  category_tags              object  
 18  geometry                   geometry
dtypes: float64(4)

In [70]:
# limit to Illinois (5.3M --> 200K places)
place_gdf_IL = place_gdf[place_gdf.region == 'IL']
place_gdf_IL.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 200614 entries, 7 to 5284977
Data columns (total 19 columns):
 #   Column                     Non-Null Count   Dtype   
---  ------                     --------------   -----   
 0   safegraph_place_id         200614 non-null  object  
 1   parent_safegraph_place_id  24590 non-null   object  
 2   location_name              200614 non-null  object  
 3   safegraph_brand_ids        47267 non-null   object  
 4   brands                     47267 non-null   object  
 5   top_category               198532 non-null  object  
 6   sub_category               198532 non-null  object  
 7   naics_code                 198532 non-null  float64 
 8   latitude                   200614 non-null  float64 
 9   longitude                  200614 non-null  float64 
 10  street_address             200614 non-null  object  
 11  city                       200614 non-null  object  
 12  region                     200614 non-null  object  
 13  posta

In [71]:
place_gdf_IL.crs

<Projected CRS: EPSG:3528>
Name: NAD83(NSRS2007) / Illinois East
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: USA - Illinois - SPCS - E
- bounds: (-89.28, 37.06, -87.02, 42.5)
Coordinate Operation:
- name: SPCS83 Illinois East zone (meters)
- method: Transverse Mercator
Datum: NAD83 (National Spatial Reference System 2007)
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

## Get the census block groups of NE Illinois

FIPS code for the counties
* Cook: 031
* Dupage: 043
* Grundy: 063
* Kane: 089
* Kankakee: 091
* Kendall: 093
* Lake: 097
* McHenry: 111
* Will: 197

In [72]:
illinois_cbg = gpd.read_file('https://raw.githubusercontent.com/loganpowell/census-geojson/master/GeoJSON/500k/2019/17/block-group.json', 
                   crs = from_epsg(2163)).to_crs(epsg = 3528)

In [73]:
# limit to 9 counties mentioned above
target_cbg = illinois_cbg[illinois_cbg.COUNTYFP.isin(['031', '043', '063', '089', '091', '093', '097', '111', '197'])]
target_cbg = target_cbg[['GEOID', 'geometry']]
target_cbg.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 5998 entries, 0 to 9688
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   GEOID     5998 non-null   object  
 1   geometry  5998 non-null   geometry
dtypes: geometry(1), object(1)
memory usage: 140.6+ KB


In [74]:
print(target_cbg.crs)
#target_cbg.head()
print(place_gdf_IL.crs)

epsg:3528
epsg:3528


In [76]:
place_gdf_target = gpd.sjoin(place_gdf_IL, target_cbg, how="inner", op="within").drop("index_right", axis = 1)
place_gdf_target.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 132091 entries, 7 to 5276130
Data columns (total 20 columns):
 #   Column                     Non-Null Count   Dtype   
---  ------                     --------------   -----   
 0   safegraph_place_id         132091 non-null  object  
 1   parent_safegraph_place_id  18968 non-null   object  
 2   location_name              132091 non-null  object  
 3   safegraph_brand_ids        30605 non-null   object  
 4   brands                     30605 non-null   object  
 5   top_category               130284 non-null  object  
 6   sub_category               130284 non-null  object  
 7   naics_code                 130284 non-null  float64 
 8   latitude                   132091 non-null  float64 
 9   longitude                  132091 non-null  float64 
 10  street_address             132091 non-null  object  
 11  city                       132091 non-null  object  
 12  region                     132091 non-null  object  
 13  posta

In [82]:
naics_summary = place_gdf_target.naics_code.value_counts()
naics_summary.head(20)

722511.0    13331
621111.0     6470
812112.0     5309
621210.0     4653
813110.0     4193
722513.0     3958
712190.0     3918
524210.0     3847
713940.0     3787
611110.0     3428
722515.0     3415
522110.0     3215
811111.0     2834
624410.0     2694
447110.0     2415
445110.0     2232
621330.0     2016
812199.0     1939
812113.0     1706
621310.0     1557
Name: naics_code, dtype: int64

In [83]:
cbg_summary = place_gdf_target.GEOID.value_counts()
cbg_summary.head(20)

170318391001    1237
170318046033     709
170313204001     505
170313201002     393
170313201001     381
170318015001     358
170438446011     322
170938901011     285
170318069001     284
170898526013     272
170310817002     264
170318330001     264
170318422001     260
170318391002     258
171118713064     256
170317707001     253
170310814011     251
170438465042     243
170318016011     237
170438465192     236
Name: GEOID, dtype: int64

### Uploading a data file from the local computer

The SafeGraph data agreement prohibits me from sharing the raw data.

In [ ]:
# upload a data file from the local computer
from google.colab import files
uploaded = files.upload()

Saving 2020-04-30-social-distancing.csv.gz to 2020-04-30-social-distancing.csv.gz


In [ ]:
!ls

2020-04-30-social-distancing.csv.gz  sample_data


### Turning device counts into the probabilities

I just use one file here, `2020-04-30-social-distancing.csv.gz`, to describe what I did.

For social distancing metrics, I focused on the `buckted_percentage_time_home` and `completely_home_device_count`.

These variables are explained in [the SafeGraph page](https://docs.safegraph.com/docs/social-distancing-metrics).
* `completely_home_device_count`: Out of the device_count, the number of devices which did not leave the geohash-7 in which their home is located during the time period.
* `buckted_percentage_time_home`: Key is range of minutes and value is device count of devices that dwelled at geohash-7 of home for the given time period, `{"<60": 0, "61-360": 0, "361-720": 10, "721-1080": 40, ">1081": 50}`. For each device, we summed the observed minutes at home across the day (whether or not these were contiguous) to get the total minutes for each device this day. Then we count how many devices are in each bucket. Beginning in v2, we include the portion of any stop within the time range regardless of whether the stop start time was in the time period.


In [ ]:
to_lit = lambda x : ast.literal_eval(x) if x is not np.nan else {}

def get_prob(input_df):

    # process the date and also get weekday
    for d in ["date_range_start", "date_range_end"]:
        input_df[d] = pd.to_datetime(input_df[d])

    input_df["date"] = pd.to_datetime(input_df["date_range_start"], utc = True)\
                   .dt.tz_convert(pytz.timezone('US/Central')).dt.date
    input_df["weekday"] = input_df["date"].apply(lambda x: x.weekday())
    
    # I focused on `buckted_percentage_time_home`, along with `completely_home_device_count`
    input_df["bucketed_percentage_time_home"] = input_df["bucketed_percentage_time_home"].apply(to_lit)
    input_df["cnt_lt_6h_out"] = input_df.bucketed_percentage_time_home.apply(lambda x : x["76-100"] if "76-100" in x else 0)
    input_df["cnt_6h_12h_out"] = input_df.bucketed_percentage_time_home.apply(lambda x : x["51-75"] if "51-75" in x else 0)
    input_df["cnt_12h_18h_out"] = input_df.bucketed_percentage_time_home.apply(lambda x : x["26-50"] if "26-50" in x else 0)
    input_df["cnt_ht_18h_out"] = input_df.bucketed_percentage_time_home.apply(lambda x : x["0-25"] if "0-25" in x else 0)

    # device counts were turned into these probabilities
    input_df["p_completely_home"] = input_df.completely_home_device_count / input_df.device_count
    input_df["p_lt_6h_out"] = input_df["cnt_lt_6h_out"] / (input_df["cnt_lt_6h_out"] + input_df["cnt_6h_12h_out"] + input_df["cnt_12h_18h_out"] + input_df["cnt_ht_18h_out"])
    input_df["p_6h_12h_out"] = input_df["cnt_6h_12h_out"] / (input_df["cnt_lt_6h_out"] + input_df["cnt_6h_12h_out"] + input_df["cnt_12h_18h_out"] + input_df["cnt_ht_18h_out"])
    input_df["p_ht_12h_out"] = 1 - (input_df["p_lt_6h_out"] + input_df["p_6h_12h_out"])

    input_df["p_part_time_work"] = input_df.part_time_work_behavior_devices / input_df.device_count
    input_df["p_full_time_work"] = input_df.full_time_work_behavior_devices / input_df.device_count
    input_df["p_delivery"] = input_df.delivery_behavior_devices / input_df.device_count
    
    select_columns = ['origin_census_block_group', 'date', 'device_count', 'distance_traveled_from_home', 
                      'p_completely_home', 'p_lt_6h_out', 'p_6h_12h_out', 'p_ht_12h_out', 
                      'p_part_time_work', 'p_full_time_work', 'p_delivery']

    output_df = input_df[select_columns]
    
    return output_df
    

In [ ]:
oneday_df = pd.read_csv('./2020-04-30-social-distancing.csv.gz')
oneday_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 219343 entries, 0 to 219342
Data columns (total 20 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   origin_census_block_group                   219343 non-null  int64  
 1   date_range_start                            219343 non-null  object 
 2   date_range_end                              219343 non-null  object 
 3   device_count                                219343 non-null  int64  
 4   distance_traveled_from_home                 219324 non-null  float64
 5   bucketed_distance_traveled                  219343 non-null  object 
 6   median_dwell_at_bucketed_distance_traveled  219324 non-null  object 
 7   completely_home_device_count                219343 non-null  int64  
 8   median_home_dwell_time                      219343 non-null  int64  
 9   bucketed_home_dwell_time                    219343 non-null  object 
 

In [ ]:
# select Cook county
oneday_df = oneday_df[oneday_df.origin_census_block_group // 10000000 == 17031]
oneday_df['GEOID'] = oneday_df.origin_census_block_group.astype('str')

# filter only Chicago CBGs
oneday_df = pd.merge(chi_cbg, oneday_df,
                  left_on = 'GEOID', right_on = 'GEOID', how = 'inner')
oneday_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 2315 entries, 0 to 2314
Data columns (total 26 columns):
 #   Column                                      Non-Null Count  Dtype   
---  ------                                      --------------  -----   
 0   GEOID                                       2315 non-null   object  
 1   geometry                                    2315 non-null   geometry
 2   name                                        2315 non-null   object  
 3   objectid                                    2315 non-null   object  
 4   shape_area                                  2315 non-null   object  
 5   shape_len                                   2315 non-null   object  
 6   origin_census_block_group                   2315 non-null   int64   
 7   date_range_start                            2315 non-null   object  
 8   date_range_end                              2315 non-null   object  
 9   device_count                                2315 non-null   int64 

In [ ]:
import warnings; warnings.simplefilter('ignore')

oneday_df2 = get_prob(oneday_df)
oneday_df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2315 entries, 0 to 2314
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   origin_census_block_group    2315 non-null   int64  
 1   date                         2315 non-null   object 
 2   device_count                 2315 non-null   int64  
 3   distance_traveled_from_home  2314 non-null   float64
 4   p_completely_home            2315 non-null   float64
 5   p_lt_6h_out                  2315 non-null   float64
 6   p_6h_12h_out                 2315 non-null   float64
 7   p_ht_12h_out                 2315 non-null   float64
 8   p_part_time_work             2315 non-null   float64
 9   p_full_time_work             2315 non-null   float64
 10  p_delivery                   2315 non-null   float64
dtypes: float64(8), int64(2), object(1)
memory usage: 217.0+ KB


### These columns are saved into social_distancing_17031.csv from 2019-01-01 to 2020-05-26